In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
import numpy as np
import os
from tensorflow.keras.utils import load_img, img_to_array

# Define organ names
organ_names = ['heart', 'brain', 'liver', 'limbs']
model = load_model(os.path.join('save_model', 'organsClssify.h5'))

# Define a label map for classification if needed
label_map = {
    0: 1,  # Map heart to brain
    1: 0,  # Map brain to heart
    2: 3,  # Map limbs to liver
    3: 2  # Map liver to limbs
}

# Initialize the path for the uploaded image
uploaded_image_path = ""


# Function to preprocess the image and make predictions
def preprocess_image(image_path, img_size=(256, 256)):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)


# Function to predict the organ from the image
def test_model(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    remapped_class = label_map.get(predicted_class, predicted_class)
    return organ_names[remapped_class]


# Function to open a dialog box to select an image
def upload_image():
    global uploaded_image_path
    uploaded_image_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if uploaded_image_path:
        try:
            img = Image.open(uploaded_image_path)
            img.thumbnail((250, 250))
            img_tk = ImageTk.PhotoImage(img)
            image_label.config(image=img_tk)
            image_label.image = img_tk

            prediction_label.config(text="")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")


# Function to predict the organ when the "Predict" button is clicked
def predict():
    if uploaded_image_path:
        try:
            # Get the actual organ name from the uploaded image's folder
            actual_organ = os.path.basename(os.path.dirname(uploaded_image_path))

            # Predict the organ using the model
            predicted_organ = test_model(uploaded_image_path)
            prediction_label.config(text=f"Predicted Organ: {predicted_organ}")

            # Calculate accuracy for this single prediction
            accuracy = 100.0 if predicted_organ == actual_organ else 0.0
            messagebox.showinfo("Accuracy", f"Accuracy for the image: {accuracy:.2f}%")

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
    else:
        messagebox.showwarning("Warning", "Please upload an image first!")


# Set up the GUI
root = tk.Tk()
root.title("Organ Classification")
root.geometry("450x600")  # Adjusted height after adding accuracy section
root.config(bg="#1e1e1e")

# Heading label
heading = tk.Label(root, text="Organ Classification", bg="#1e1e1e", fg="#ffffff", font=("Arial", 16, "bold"))
heading.pack(pady=10)

# Frame for image display
frame = tk.Frame(root, bg="#2e2e2e", padx=20, pady=20, relief="groove", bd=2)
frame.pack(pady=10)

# Label to display the image
image_label = tk.Label(frame, bg="#2e2e2e")
image_label.pack(pady=10)

# Button to upload image
upload_button = tk.Button(root, text="Upload Image", command=upload_image, bg="#00796b", fg="white", font=("Arial", 12),
                          padx=10)
upload_button.pack(pady=10)

# Button to predict the organ
predict_button = tk.Button(root, text="Predict", command=predict, bg="#e91e63", fg="white", font=("Arial", 12), padx=10)
predict_button.pack(pady=10)

# Label to display the prediction result
prediction_label = tk.Label(root, text="", bg="#1e1e1e", fg="#b0b0b0", font=("Arial", 14))
prediction_label.pack(pady=10)

# Start the GUI loop
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
